In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.gridspec import GridSpec

import pyarrow as pa
import pyarrow.parquet as pq

import seaborn as sns
pd.set_option('display.max_columns', None)

In [ ]:
dates_analysis = pd.read_csv('sorted_ids_06_FEB_2024.csv')
#dates_analysis

In [ ]:
ids = np.load('ids_seven_days_each_five_bimonth.npy')


patient = pd.read_parquet('Patients_lookup.parquet', engine='pyarrow')#dcarte.load('Patients','lookup',reload=False)
patient['participant_id'] = patient['id'].apply(lambda x:x[:5])
patient = patient[patient['participant_id'].isin(ids)]

dyads = pd.read_excel('dyads_unprotected.xlsx')

dyads['participant_id'] = dyads['Research ID'].apply(lambda x:x[:5])
dyads = dyads[dyads['participant_id'].isin(ids)]

adas = pd.read_csv('redcap_16_nov_2023/MinderHealthManageme_DATA_2024-02-12_1458.csv')
                 #  MinderHealthManageme_DATA_LABELS_2024-02-09_1330.csv')
#dyads[['DYAD','participant_id']]
#print(dyads['DYAD'].unique())
adas.columns

In [ ]:
adas = adas[(adas['total_score_adascog'].notnull())&adas['date_adascog'].notnull()].reset_index(drop=True)# 

#adas  = adas.rename(columns={"participant's ID": "DYAD"})
adas  = adas.rename(columns={"id_adascog": "DYAD"})

adas = adas[adas['DYAD']!='TU12'].reset_index(drop=True)
print(len(adas))

adas['DYAD'] = pd.to_numeric(adas['DYAD'])
print(len(adas),len(adas['DYAD'].unique()))
#adas[['DYAD','Test date','Total Score']]
adas[['DYAD','date_adascog','total_score_adascog']]

In [ ]:
#adas = adas[['DYAD','Test date','Total Score']].merge(dyads[['DYAD','participant_id']], on='DYAD')
adas = adas[['DYAD','date_adascog','total_score_adascog']].merge(dyads[['DYAD','participant_id']], on='DYAD')
adas = adas[['participant_id','date_adascog','total_score_adascog']]
print(len(adas),len(adas['participant_id'].unique()))
adas['date'] = pd.to_datetime(adas['date_adascog'])
adas

In [ ]:
adas = adas.merge(dates_analysis[['participant_id','min_date', 'max_date']],
                  on='participant_id', how='left')
adas

In [ ]:
print(len(adas),len(adas['participant_id'].unique()))

In [ ]:
adas['time_in_study'] = (adas['date']-pd.to_datetime(adas['min_date'])).dt.days
adas['days_after_max_date'] = (adas['date']-pd.to_datetime(adas['max_date'])).dt.days
adas = adas[(adas['time_in_study']>=0)&(adas['days_after_max_date']<=0)].reset_index(drop=True)
adas['month'] = pd.to_datetime(adas['date']).dt.strftime('%B')#.month
adas['bimonth'] ='JanFeb'
adas.loc[(adas['month']=='March')|(adas['month']=='April'), 'bimonth'] = 'MarApr'
adas.loc[(adas['month']=='May')|(adas['month']=='June'), 'bimonth'] = 'MayJun'
adas.loc[(adas['month']=='July')|(adas['month']=='August'), 'bimonth'] = 'JulAug'
adas.loc[(adas['month']=='September')|(adas['month']=='October'), 'bimonth'] = 'SepOct'
adas.loc[(adas['month']=='November')|(adas['month']=='December'), 'bimonth'] = 'NovDec'

adas.columns

In [ ]:
br  .to_csv('npi_values_30_NOV_23_for_lme.csv')

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(adas['total_score_adascog'], bins=range(0,100,5),rwidth=0.9, color='tomato')


#plt.yticks([1,3,5,7,9], rotation=0)
#plt.xticks([0.5,5.5,10.5,15.5,20.5,25.5,30.5],[0,5,10,15,20,25,30], rotation=0)
plt.xlabel('ADAS-Cog Score Per Participant')
plt.show()

In [ ]:
adas['date'].min(), adas['date'].max()

In [ ]:
count = adas.groupby('participant_id').count().reset_index()
plt.figure(figsize=(6,4))
plt.hist(count['date'], bins=range(0,8,1),rwidth=0.9, color='tomato')


#plt.yticks([1,3,5,7,9], rotation=0)
plt.xticks([0.5,1.5,2.5,3.5,4.5,5.5,6.5],[0,1,2,3,4,5,6], rotation=0)
plt.xlabel('Number ADAS-Cog Tests Per Participant')
plt.show()

In [ ]:
count = adas.groupby('participant_id').mean().reset_index()
count
plt.figure(figsize=(6,4))
plt.hist(count['total_score_adascog'], bins=range(0,80,4),rwidth=0.9, color='tomato')


#plt.yticks([1,3,5,7,9], rotation=0)
#plt.xticks([0.5,1.5,2.5,3.5,4.5,5.5,6.5],[0,1,2,3,4,5,6], rotation=0)
plt.xlabel('Average ADAS-Cog Tests Per Participant')
plt.show()